## **演示1103：处理连续型特征值**

### **提出问题**
通过一些测量的特征，包括身高、体重、脚的尺寸，判定一个人是男性还是女性。
已有训练数据如下表：

![](../images/110301.png)

与之前例子中的单词个数统计不同，身高、体重、脚的尺寸等数据是连续分布的，无法用计数来统计。例如，分别统计身高为6、6.01、6.02、6.03....的人所占的比重，是没有意义的。因此，对于连续型随机变量，无法像离散型变量一样使用$\dfrac{T_(x_i|c_k)}{T_{c_k}} $来计算每个Feature在某个分类中的概率。

### **分析问题**
* 连续型随机变量对应有概率密度函数，可以通过概率密度函数来计算在某一点的概率密度（注意，连续型随机变量的PDF计算出来的某一点的值，并不是直接的概率）。我们将**采用概率密度函数值代替离散情形下的概率值**。此时的朴素贝叶斯公式变为：  
$ P(y=c_k |x)= \dfrac{pdf(x_1 |y=c_k) * pdf(x_2 |y=c_k) * \cdots * pdf(x_n |y=c_k) * P(y=c_k)}{evidence} $
 * 分母evidence(全概率)一般不必计算，只需比较不同分类中分子的大小，就能判断出所属的类别
 * 即使是连续型随机变量，每种分类所占的比重$P(y=c_k)$仍然是可以计算的。例如，我们可以从训练样本数据种计算出本例中两个类别(男、女)的概率为$P(y=c_1 )=P(y=c_2 )=0.5$
 * 只要能够确定概率密度函数pdf，就能够计算出后验概率
* 在大部分情况下，可以假定随机变量的分布遵循正态分布。因此我们可以直接采用正态分布的概率密度函数。**正态分布的pdf需要指定均值$\mu$和方差$\sigma^2$，这两个参数可以根据训练数据计算出来**，例如：  
$ \mu(x_1| y=1)=\dfrac{\sum_{i=1}^{m_{(y=1)}} x_1^{(i)}}{m_{(y=1)}} = (6+5.92+5.58+5.92)/4=5.855 $  
$ \begin{aligned}
\sigma_{(x_1 | y=1)}^2=&\dfrac{\sum_{i=1}^{m_{(y=1)}} [x_1^{(i)} -\mu(x_1 | y=1)]^2}{m_{(y=1)}-1} \\ =&
((6-5.855)^2+(5.92-5.855)^2+(5.58-5.855)^2+(5.92-5.855)^2)/3 \\
=&0.03503
\end{aligned}$  
 * $y=1$代表分类为"男"的情形，$y=0$代表分类为"女"的情形
 * $m_{(y=1)}$ 表示$y=1$情形下的数据条数  
按照这种方法，可以依次计算出每个Feature在每种分类下的正态分布的$\mu$和$\sigma^2$：  
![](../images/110302.png)
* 使用正态分布的概率密度函数计算后验概率。例如，要求身高、体重、脚尺寸分别为6、130、8的测试样本所述的分类，可以这么做：  
$ pdf(x_1|y=1)=\dfrac{1}{ {\sigma_{x_1|y=1}} * {\sqrt {2 \pi}} } * e^{-\dfrac{(x-\mu(x_1|y=1))^2}{2 {\sigma_{x_1|y=1}}^2}}= \dfrac{1}{\sqrt {0.035033} * \sqrt{2 \pi}} * e^{-\dfrac{(6-5.855)^2}{2 * 0.035033}} =1.5789 $  
依次类推，计算出$pdf(x_2|y=1)$和$pdf(x_3 |y=1)$，再结合$P(y=1)=0.5$，可计算出：  
$ P(y=1|x)=\dfrac{6.1984^{-9}}{evidence} $  
$ P(y=2|x)=\dfrac{5.3778^{-4}}{evidence} $  
可以看出，该测试样本是女性的概率要远远高于男性

### **案例1：使用sklearn.naive_bayes.GaussianNB对连续性随机变量进行朴素贝叶斯分类**

In [1]:
''' 使用贝叶斯回归处理连续特征值 '''

import numpy as np
from sklearn.naive_bayes import GaussianNB

training_feature = [[6,180,12],[5.92,190,11],[5.58,170,12],[5.92,165,10],
                    [5,100,6],[5.5,150,8],[5.42,130,7],[5.75,150,9]]
training_classify = [1,1,1,1,0,0,0,0]

model = GaussianNB()
model.fit(training_feature, training_classify)

testing_feature = [[6,130,8],[5.9,177,10]]
predict_classify = model.predict(testing_feature)
print(predict_classify)

[0 1]
